In [1]:
#%load_ext autoreload
#%autoreload 2

from layers import *
from model import *
from data import *

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [14]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)

# Pairs of scale fact and group invariance...

tests = [(1, IdentityGroup, "Identity"),
         (2, IdentityGroup, "Identity"),
         
         (1, HorizontalSymGen, "HorizontalFlips"),
         (2, HorizontalSymGen, "HorizontalFlips"),
         (4, HorizontalSymGen, "HorizontalFlips"),
         
         (1, FlipsSymGen,      "Flips"),
         (2, FlipsSymGen,      "Flips"),
         (4, FlipsSymGen,      "Flips"),
         
         (2, RotFlipSymGen,    "RotFlips"),
         (4, RotFlipSymGen,    "RotFlips"),
         (8, RotFlipSymGen,    "RotFlips"),
         ]

for t in tests:
    sf, group, name = t
    
    model = unet_sym(None, (256,256,1), sf, group)
        
    # Number of weights in model
    trainable_count = int(
        np.sum([K.count_params(p) for p in list(model.trainable_weights)]))
    
    mod_name = "UnetSym-" + name + "-sf" + str(sf) + "-weights" + str(trainable_count)
    print("Now training:", mod_name)
    
    # Callback to log to tensorboard...
    log_dir = 'logs/scalars/' + mod_name
    tf_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print("!!!!!!!!!!!!! Logging to", log_dir, "!!!!!!!!!!!!!!!!!!")

    #model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    model.fit_generator(myGene, steps_per_epoch=300, epochs=10, callbacks=[tf_callback])

lft 1
Now training: UnetSym-Identity-sf1-weights31031685
!!!!!!!!!!!!! Logging to logs/scalars/UnetSym-Identity-sf1-weights31031685 !!!!!!!!!!!!!!!!!!
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


W0226 15:51:23.688531 47205104539648 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']


Train for 300 steps


W0226 15:51:24.032775 47205104539648 summary_ops_v2.py:1132] Model failed to serialize as JSON. Ignoring... Layers with arguments in `__init__` must override `get_config`.


Epoch 1/10
300/300 [==============================] - 133s 445ms/step - loss: 0.3972 - accuracy: 0.8021
Epoch 2/10
 48/300 [===>..........................] - ETA: 1:50 - loss: 0.3407 - accuracy: 0.8429

KeyboardInterrupt: 

In [ ]:
# Equivarance tests
#model = unet_sym(None, (128,128,1), 4, RotFlipSymGen)

a = np.random.rand(1,256,256,1)

def diff(a, b):
    # Print maximum abs difference.
    print(np.amax(np.abs(a-b)), flush=True)
    
print("sanity:")
diff(model.predict(a), rot(model.predict(a)))
diff(model.predict(a), model.predict(rot(a))) # Sanity checks, should be different.
print("rot")
diff(rot(model.predict(a)), model.predict(rot(a, 1))) # Rotation equivarience
print("flips")
diff(flipH(model.predict(a)), model.predict(flipH(a))) # Horizontal flip
diff(flipV(model.predict(a)), model.predict(flipV(a))) # Vertical flip

In [ ]:
# Base model; 32M weights:
# Train for 300 steps
# Epoch 1/12
# 300/300 [==============================] - 26s 88ms/step - loss: 0.3354 - accuracy: 0.8538
# Epoch 2/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.2732 - accuracy: 0.9015
# Epoch 3/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.2510 - accuracy: 0.9132
# Epoch 4/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.2384 - accuracy: 0.9191
# Epoch 5/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.2286 - accuracy: 0.9236
# Epoch 6/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.2202 - accuracy: 0.9277
# Epoch 7/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.2147 - accuracy: 0.9304
# Epoch 8/12
# 300/300 [==============================] - 24s 82ms/step - loss: 0.2092 - accuracy: 0.9330
# Epoch 9/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.2029 - accuracy: 0.9362
# Epoch 10/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.1979 - accuracy: 0.9386
# Epoch 11/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.1924 - accuracy: 0.9410
# Epoch 12/12
# 300/300 [==============================] - 24s 81ms/step - loss: 0.1884 - accuracy: 0.9430

# Symnet: scaled down by 2; 15M weights
# Train for 300 steps
# Epoch 1/12
# 300/300 [==============================] - 45s 151ms/step - loss: 0.3701 - accuracy: 0.8333
# Epoch 2/12
# 300/300 [==============================] - 36s 120ms/step - loss: 0.2775 - accuracy: 0.8830
# Epoch 3/12
# 300/300 [==============================] - 36s 121ms/step - loss: 0.2658 - accuracy: 0.8870
# Epoch 4/12
# 300/300 [==============================] - 36s 121ms/step - loss: 0.2581 - accuracy: 0.8919
# Epoch 5/12
# 300/300 [==============================] - 36s 121ms/step - loss: 0.2172 - accuracy: 0.9085
# Epoch 6/12
# 300/300 [==============================] - 37s 122ms/step - loss: 0.2031 - accuracy: 0.9138
# Epoch 7/12
# 300/300 [==============================] - 36s 121ms/step - loss: 0.1855 - accuracy: 0.9200
# Epoch 8/12
# 300/300 [==============================] - 36s 121ms/step - loss: 0.1869 - accuracy: 0.9199
# Epoch 9/12
# 300/300 [==============================] - 37s 122ms/step - loss: 0.1733 - accuracy: 0.9247
# Epoch 10/12
# 300/300 [==============================] - 36s 122ms/step - loss: 0.1717 - accuracy: 0.9257
# Epoch 11/12
# 300/300 [==============================] - 37s 122ms/step - loss: 0.1619 - accuracy: 0.9293
# Epoch 12/12
# 300/300 [==============================] - 36s 122ms/step - loss: 0.1558 - accuracy: 0.9318

# Symnet: Full scale; 62M weights
# Train for 300 steps
# Epoch 1/12
# 300/300 [==============================] - 100s 332ms/step - loss: 0.3495 - accuracy: 0.8527
# Epoch 2/12
# 300/300 [==============================] - 95s 315ms/step - loss: 0.2097 - accuracy: 0.9097
# Epoch 3/12
# 300/300 [==============================] - 95s 315ms/step - loss: 0.1750 - accuracy: 0.9241
# Epoch 4/12
# 300/300 [==============================] - 95s 315ms/step - loss: 0.1553 - accuracy: 0.9322
# Epoch 5/12
# 300/300 [==============================] - 95s 315ms/step - loss: 0.1402 - accuracy: 0.9387
# Epoch 6/12
# 300/300 [==============================] - 95s 315ms/step - loss: 0.1258 - accuracy: 0.9451
# Epoch 7/12
# 300/300 [==============================] - 95s 315ms/step - loss: 0.1153 - accuracy: 0.9497
# Epoch 8/12
# 300/300 [==============================] - 94s 315ms/step - loss: 0.1070 - accuracy: 0.9534
# Epoch 9/12
# 300/300 [==============================] - 95s 315ms/step - loss: 0.0999 - accuracy: 0.9566
# Epoch 10/12
# 300/300 [==============================] - 94s 315ms/step - loss: 0.0946 - accuracy: 0.9590
# Epoch 11/12
# 300/300 [==============================] - 94s 315ms/step - loss: 0.0896 - accuracy: 0.9612
# Epoch 12/12
# 300/300 [==============================] - 94s 315ms/step - loss: 0.0860 - accuracy: 0.9628

### Train with npy file

In [ ]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [ ]:
testGene = testGenerator("data/membrane/test")
#model = unet()
#model.load_weights("unet_membrane.hdf5")
model.summary()
results = model.predict_generator(testGene,30,verbose=1)
saveResult("data/membrane/test",results)